In [51]:
import pandas as pd

# Step 1 Load Data
data = pd.read_csv('ugly_sweaters.csv')
data = data[data['hs_tf'] == 'Yes']
print(data)

        sweater hs_tf sparkly noise lights objects  \
0      sweater1   Yes     Yes    No     No      No   
1      sweater2   Yes      No    No     No      No   
2      sweater3   Yes      No    No     No      No   
4      sweater5   Yes      No    No     No      No   
6      sweater8   Yes      No   Yes    Yes     Yes   
..          ...   ...     ...   ...    ...     ...   
97   sweater100   Yes     Yes    No     No     Yes   
100  sweater103   Yes      No   Yes     No     Yes   
101  sweater104   Yes     Yes    No     No     Yes   
102  sweater105   Yes      No    No     No      No   
104  sweater107   Yes      No    No     No     Yes   

                                      colors image_tf  \
0             Red, Yellow, Blue, White, teal      Yes   
1                                      Green       No   
2    Red, Yellow, Green, Brown, White, Black      Yes   
4                         Blue, White, Black      Yes   
6      Red, Green, Blue, Purple, White, Grey       No   
..       

In [52]:
# Step 2 Wrangle Data
data['colors'] = data['colors'].str.split(',')
expanded_data = data.explode('colors')
expanded_data = expanded_data.groupby('sweater').count()
data.index = expanded_data.index
data['num_colors'] = expanded_data['hs_tf']
print(data)


               sweater hs_tf sparkly noise lights objects  \
sweater                                                     
sweater1      sweater1   Yes     Yes    No     No      No   
sweater100    sweater2   Yes      No    No     No      No   
sweater103    sweater3   Yes      No    No     No      No   
sweater104    sweater5   Yes      No    No     No      No   
sweater105    sweater8   Yes      No   Yes    Yes     Yes   
...                ...   ...     ...   ...    ...     ...   
sweater94   sweater100   Yes     Yes    No     No     Yes   
sweater96   sweater103   Yes      No   Yes     No     Yes   
sweater97   sweater104   Yes     Yes    No     No     Yes   
sweater98   sweater105   Yes      No    No     No      No   
sweater99   sweater107   Yes      No    No     No     Yes   

                                                    colors image_tf  \
sweater                                                               
sweater1              [Red,  Yellow,  Blue,  White,  teal]      